In [1]:
import pandas as pd

In [2]:
tumor_mat = pd.read_csv("../data_linkedomics/N-glycoproteomics_peptide_level_ratio_tumor.cct", sep="\t")
tumor_mat["Gene"] = tumor_mat["Gene"].str.split("-").str[0]
normal_mat = pd.read_csv("../data_linkedomics/N-glycoproteomics_peptide_level_ratio_normal.cct", sep="\t")
normal_mat["Gene"] = normal_mat["Gene"].str.split("-").str[0]
tumor_mat = tumor_mat.set_index(["Gene","Sequence"]).sort_index()
normal_mat = normal_mat.set_index(["Gene","Sequence"]).sort_index()


cnt_df = pd.DataFrame({
    "tumor": tumor_mat.notna().sum(axis=1),
    "normal": normal_mat.notna().sum(axis=1)
})

tumor_mat["non_NA_count"] = tumor_mat.notna().sum(axis=1)
normal_mat["non_NA_count"] = normal_mat.notna().sum(axis=1)
cnt_df["total"] = cnt_df["tumor"] + cnt_df["normal"]
tumor_mat.head()
cnt_df.head(10)

tumor  normal  total
Gene Sequence                                                          
A1BG EGDHEFLEVPEAQEDVEATFPVHQPGNYSCSYR-N4H5F0S0G0     27      11     38
     EGDHEFLEVPEAQEDVEATFPVHQPGNYSCSYR-N4H5F0S1G0     90      46    136
     EGDHEFLEVPEAQEDVEATFPVHQPGNYSCSYR-N4H5F0S2G0    138      74    212
     EGDHEFLEVPEAQEDVEATFPVHQPGNYSCSYR-N4H5F1S2G0      7       3     10
     EGDHEFLEVPEAQEDVEATFPVHQPGNYSCSYR-N4H5F2S0G0     44      21     65
     EGDHEFLEVPEAQEDVEATFPVHQPGNYSCSYR-N4H5F2S1G0    106      56    162
     EGDHEFLEVPEAQEDVEATFPVHQPGNYSCSYR-N4H6F1S1G0     13       5     18
     EGDHEFLEVPEAQEDVEATFPVHQPGNYSCSYR-N4H6F2S0G0      0       7      7
     EGDHEFLEVPEAQEDVEATFPVHQPGNYSCSYR-N4H6F3S0G0      8       2     10
     EGDHEFLEVPEAQEDVEATFPVHQPGNYSCSYR-N5H6F0S0G0      6       4     10

In [4]:
cnt_df[(cnt_df["tumor"]>3) & (cnt_df["normal"]>3)].index.get_level_values(0).unique()

Index(['A1BG', 'A2M', 'A2ML1', 'AADAC', 'ABCA1', 'ABCA2', 'ABCA6', 'ABCA8',
       'ABI3BP', 'ACAN',
       ...
       'VWF', 'WFDC2', 'WFS1', 'WNT11', 'WNT5A', 'YBX3', 'YWHAE', 'ZFYVE26',
       'ZNF501', 'ZPBP'],
      dtype='object', name='Gene', length=1037)

In [ ]:
# peptide="PLVAP"
# tumor_mat.loc[peptide]
# normal_mat.loc[peptide]
# cnt_df.loc[peptide]
# for i in cnt_df[(cnt_df["tumor"]>=3) & (cnt_df["normal"]>=3)].index.get_level_values(0).unique():
    # print(i)

# cnt_df

,tumor,normal,total
Sequence,,,
RAEGLYSQLLGLTASQSNLTK-N6H7F4S1G0,0,5,5
RAEGLYSQLLGLTASQSNLTK-N6H7F5S0G0,0,7,7


In [113]:
df_sum = cnt_df.groupby(level=0).sum()
df_sum[df_sum["total"]>=100]

,tumor,normal,total
Gene,,,
A1BG,1527,803,2330
A2M,5734,2919,8653
A2ML1,284,124,408
AADAC,194,108,302
ABCA1,282,140,422
...,...,...,...
WFDC2,100,50,150
WFS1,464,262,726
WNT11,190,94,284


In [ ]:


# tumor_mat[tumor_mat["non_NA_count"]>=5].index.get_level_values(0).unique()

Index(['A1BG', 'A2M', 'A2ML1', 'AADAC', 'ABCA1', 'ABCA2', 'ABCA6', 'ABCA8',
       'ABCC4', 'ABCC5',
       ...
       'WFDC2', 'WFS1', 'WNT11', 'WNT5A', 'YBX3', 'YTHDF3', 'YWHAE', 'ZFYVE26',
       'ZNF501', 'ZPBP'],
      dtype='object', name='Gene', length=1106)

In [35]:
# for i in (cnt_df[(cnt_df["tumor"] >= 4) & (cnt_df["normal"]>=4)].index.get_level_values(0).unique()):
#     print(i)

# cnt_df.loc["TXNDC15"]
# tumor_mat.loc[("total",)] = tumor_mat.notna().sum()

# each value 2**x
tumor_mat_2x = tumor_mat.copy()
tumor_mat_2x.iloc[:, :-1] = tumor_mat_2x.iloc[:, :-1].applymap(lambda x: 2**x if pd.notna(x) else x)
# If multiple sequences from the same gene, sum them up, but replace 0.000 with NaN
tumor_mat_2x = tumor_mat_2x.groupby(level=0).sum()
tumor_mat_2x = tumor_mat_2x.replace(0.000, pd.NA)

# Add a row for the total sum of each column
non_na_counts = tumor_mat_2x.notna().sum(axis=0)
total_row = pd.DataFrame([non_na_counts], index=[("total",)])
tumor_mat_2x = pd.concat([tumor_mat_2x, total_row])
# Drop the 'non_NA_count' column
tumor_mat_2x = tumor_mat_2x.drop(columns=["non_NA_count"], errors='ignore')
# Count columns with total > 550
tumor_mat_2x.iloc[-1, :-1].gt(600)

/tmp/ipykernel_2511935/3460414420.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tumor_mat_2x.iloc[:, :-1] = tumor_mat_2x.iloc[:, :-1].applymap(lambda x: 2**x if pd.notna(x) else x)


C3N-03884    True
C3L-03123    True
C3L-01687    True
C3L-00589    True
C3L-00599    True
             ... 
C3N-00511    True
C3L-02613    True
C3N-00512    True
C3L-02899    True
C3N-03006    True
Name: (total,), Length: 139, dtype: bool

In [91]:
df_sum = cnt_df.groupby(level=0).sum()
df_sum
# df_sum[df_sum["tumor"] >=8]
# for i in df_sum[(df_sum["total"]>=8)].index:
    # print(i)

,tumor,normal,total
Gene,,,
A1BG,1527,803,2330
A2M,5734,2919,8653
A2ML1,284,124,408
AADAC,194,108,302
ABCA1,282,140,422
...,...,...,...
YTHDF3,15,8,23
YWHAE,33,17,50
ZFYVE26,95,52,147


In [101]:
gene_level = 0  # MultiIndex에서 gene은 level 0

gene_presence = tumor_mat.groupby(level=gene_level).apply(
    lambda df: df.notna().any(axis=0)
)

# True의 개수 → 각 sample이 몇 개의 gene을 가지고 있는지
gene_counts_per_sample = gene_presence.sum(axis=0)
sum(gene_counts_per_sample > 600)
# len(gene_counts_per_sample)

131